# Lezione 6, l'algoritmo CYK


In [ ]:
import os
os.environ["ANTLR4_JAR"] = "/home/federicobruzzoneplasma/Documents/FedericoBruzzone/master-courses/linguaggi-e-traduttori/lecture/jars/antlr-4.12.0-complete.jar"
from pprint import pprint as p

In [ ]:
from itertools import count

from liblet import (
  Tree, Grammar, Production, Derivation, ProductionGraph, CYKTable, closure, union_of, ε
)

## La tabella `R` e la sua relazione con l'input

In [ ]:
INPUT = 'unaprova'

n = len(INPUT)

R = CYKTable()
for l in range(1, n + 1):
  for i in range(1, n - l + 2): 
    R[i, l] = INPUT[(i) - 1: (i + l) - 1]

R

## Due modi per riempire la tabella…

Supponendo di avere la funzione `fill(R, i, l)` che restituisce il valore di `R[(i, l)`.

In [ ]:
def offline(fill, n):
  R = CYKTable()
  for l in range(1, n + 1):
    for i in range(1, n - l + 2): 
      R[i, l] = fill(R, i, l)
  return R

In [ ]:
def online(fill, n):
  R = CYKTable()
  for d in range(1, n + 1):
    for i in range(d, 0, -1):
      R[i, d - i + 1] = fill(R, i, d - i + 1)
  return R

La seguente funzione `make_counter_fill` restituisce una funzione `fill` che, ignorando i parametri, restituisce un numero progressivo (usando `count` del modulo `itertools`).

In [ ]:
def make_counter_fill():
  cnt = count(1)
  def fill(R, i, l):
    return next(cnt)
  return fill

Usando `offline` e `online` con il contatore si visualizza l'ordine di riempimento

In [ ]:
offline(make_counter_fill(), 5)

In [ ]:
online(make_counter_fill(), 5)

## Filtrare le produzioni coi predicati

Usando la funzione *builtin* `filter` e un *predicato*  si può ottenere un sottoiteratore.

In [ ]:
def pari(x):
  return x % 2 == 0

list(filter(pari, range(10)))

Vediamo due predicati per le produzioni…

In [ ]:
prods = Production.from_string("""
A -> a
A -> B C
""")
prods

In [ ]:
# quelle con rhs lungo 2

list(filter(Production.such_that(rhs_len = 2), prods))

In [ ]:
# e quelle con rhs lungo 1

list(filter(Production.such_that(rhs_len = 1), prods))

## Il riempimento dell'algoritmo CYK nel caso CNF

Supponiamo che la grammatica sia in *Chomsky Normal Form*, ovvero le produzioni abbiano la forma $A\to BC$ o $A\to a$ (senza ε-regole).

In [ ]:
def cyk_fill(G, INPUT):
  def fill(R, i, l):
    res = set()
    if l == 1:
      for A, (a,) in filter(Production.such_that(rhs_len = 1), G.P): 
        if a == INPUT[i - 1]: res.add(A)
    else:
      for k in range(1, l):
        for A, (B, C) in filter(Production.such_that(rhs_len = 2), G.P):
          if B in R[i, k] and C in R[i + k, l - k]: res.add(A)
    return res
  return fill

Qualche esempio su una grammatica per `a`$^n$`.`

In [ ]:
G = Grammar.from_string("""
S -> A S
A -> a
S -> .
""")

In [ ]:
INPUT = 'aaa.'

online(cyk_fill(G, INPUT), len(INPUT))

In [ ]:
INPUT = 'aa.a.'

online(cyk_fill(G, INPUT), len(INPUT))

Una grammatica più complessa (per i numeri decimali con virgola, in notazione scientifica)

In [ ]:
# fig. 4.15, pag. 123 

G = Grammar.from_string("""
Number -> 0|1|2|3|4|5|6|7|8|9 
Number -> Integer Digit
Number -> N1 Scale' | Integer Fraction
N1 -> Integer Fraction
Integer -> 0|1|2|3|4|5|6|7|8|9 
Integer -> Integer Digit
Fraction -> T1 Integer
T1 -> .
Scale' -> N2 Integer
N2 -> T2 Sign
T2 -> e
Digit -> 0|1|2|3|4|5|6|7|8|9 
Sign -> + | -
""")

In [ ]:
# per comodità

def cyk(G, INPUT):
  return offline(cyk_fill(G, INPUT), len(INPUT))

In [ ]:
INPUT = '32.5e+1'

R = cyk(G, INPUT)
R

## Generare l'albero di parsing (barando)

La tabella contiene non terminali e, in accordo al criterio usato per riempirla, tali non terminali possono essere raccolti in un albero che ha l'aspetto di un albero di derivazione — anche se costruito senza riferimento alle produzioni (dal quale non è quindi banale ricostruire la derivazione).

Scriviamo una funzione ricorsiva `fake_parse` che (usando tabella `R`, la grammatica `G` e l'input `INPUT`) dato un non terminale, il punto d'inizio e la lunghezza, restituisca l'albero di parsing radicato in quel non terminale e che deriva la sottostringa specificata.

In [ ]:
def fake_parse(X, i, l):
  if l == 1: return Tree(X, [Tree(INPUT[i-1])])
  for A, (B, C) in filter(Production.such_that(lhs = X, rhs_len = 2), G.P):
    for k in range(1, l):
      if B in R[i, k] and C in R[i + k, l - k]:
        return Tree(A, [fake_parse(B, i, k), fake_parse(C, i + k, l - k)])

In [ ]:
fake_parse(G.S, 1, len(INPUT))

## Trasformazione in forma normale di Chomsky

Recuperiamo dalla lezione 3 come eliminare le regole improduttive e irraggiungibili.

In [ ]:
def remove_unproductive_unreachable(G):
  def find_productive(G):
    @closure
    def find(prod):
      return prod | {A for A, α in G.P if set(α) <= prod}
    return find(G.T)
  def find_reachable(G):
    @closure
    def find(reach):
      return reach | union_of(set(α) for A, α in G.P if A in reach)
    return find({G.S})
  Gp = G.restrict_to(find_productive(G))
  return Gp.restrict_to(find_reachable(Gp))

### Eliminazione ε-regole (Sez. 4.2.3.1)

Due passi, ottenuti tramite *chiusura*, consentono di rimpiazzare un simbolo nei lati destri con `replace_in_rhs` e quindi applicare il primo passo a tutti i simboli che compaiono in una ε-regola con `inline_ε_rules`.

In [ ]:
@closure
def replace_in_rhs(G, A):
  Ap = A + '′'
  prods = set()
  for B, β in G.P:
    if A in β:
      pos = β.index(A)
      rhs = β[:pos] + β[pos + 1:]
      if len(rhs) == 0: rhs = (ε, )
      prods.add(Production(B, rhs))
      prods.add(Production(B, β[:pos] + (Ap, ) + β[pos + 1:]))
    else:
      prods.add(Production(B, β))
  return Grammar(G.N | {Ap}, G.T, prods, G.S)

In [ ]:
# esempio d'uso

U = Grammar.from_string("""
S -> x A y A z
A -> a
""")
replace_in_rhs(U, 'A').P

In [ ]:
@closure
def inline_ε_rules(G_seen):
  G, seen = G_seen
  for A in G.N - seen:
    if (ε, ) in G.alternatives(A):
      return replace_in_rhs(G, A), seen | {A}
  return G, seen

In [ ]:
# esempio d'uso

U = Grammar.from_string("""
S -> A
A -> B C
B -> ε
C -> ε
""")
U, _ = inline_ε_rules((U, set()))

U.P

Usando i due passi precedenti è semplice scrivere il passo di eliminazione

In [ ]:
def eliminate_ε_rules(G):
  Gp, _ = inline_ε_rules((G, set()))
  prods = set(Gp.P)
  for Ap in Gp.N - G.N:
    A = Ap[:-1]
    for α in set(Gp.alternatives(A)) - {(ε, )}:
      prods.add(Production(Ap, α))
  return Grammar(Gp.N, Gp.T, prods, Gp.S)

In [ ]:
# esempio d'uso (fig. 4.10, pag. 120)

U = Grammar.from_string("""
S -> L a M
L -> L M 
L -> ε
M -> M M
M -> ε
""")

eliminate_ε_rules(U).P

### Eliminazione regole unitarie (Sez. 4.2.3.2)

In [ ]:
def eliminate_unit_rules(G):
  @closure
  def eliminate(G_seen):
    G, seen = G_seen
    for P in set(filter(Production.such_that(rhs_len = 1), G.P)) - seen:
      A, (B, ) = P
      if B in G.N:
        prods = (set(G.P) | {Production(A, α) for α in G.alternatives(B)}) - {P}
        return Grammar(G.N, G.T, prods, G.S), seen | {P}
    return G, seen
  return eliminate((G, set()))[0]

In [ ]:
# esempio d'uso

U = Grammar.from_string("""
S -> A
A -> B
B -> A | b
""")

eliminate_unit_rules(U).P

#### Un esempio più elaborato

In [ ]:
# fig. 4.6, pag. 112

G = Grammar.from_string("""
Number -> Integer | Real
Integer -> Digit | Integer Digit
Real -> Integer Fraction Scale
Fraction -> . Integer
Scale -> e Sign Integer | Empty
Digit -> 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9
Sign -> + | -
Empty -> ε
""")

In [ ]:
# fig. 4.12, pag. 120 (a meno di Empty′)

eliminate_ε_rules(G).P

Data la `G` per i numeri con la virgola, si ottiene `Gp` coi primi due passi di cui sopra.

In [ ]:
# fig. 4.13, pag. 121

Gp = eliminate_unit_rules(eliminate_ε_rules(G))
Gp.P

### Eliminare regole/simboli non produttive e  non raggiungibili (Sez. 2.9.5.1-2)

In [ ]:
# fig. 4.14, pag. 122

Gp_clean = remove_unproductive_unreachable(Gp)
Gp_clean.P

### Riduzione in forma normale (Sez. 4.3.2.4)

#### Caso $A\to \alpha a \beta$

In [ ]:
def transform_nonsolitary(G):
  prods = set()
  for A, α in G.P:
    prods.add(Production(A, [f'N{x}' if x in G.T else x for x in α] if len(α) > 1 else α))
    prods |= {Production(f'N{x}', (x, )) for x in α if x in G.T and len(α) > 1}
  return Grammar(G.N | {A for A, α in prods}, G.T, prods, G.S)

In [ ]:
# esempio d'uso

U = Grammar.from_string("""
S -> x S y S x
""")

transform_nonsolitary(U).P

#### Caso $A\to X_1 X_2\ldots X_n$

In [ ]:
def make_binary(G):
  prods = set()
  for A, α in G.P:
    if len(α) > 2:
      Ai = f'{A}{1}'
      prods.add(Production(Ai, α[:2]))
      for i, Xi in enumerate(α[2:-1], 2):
          prods.add(Production(f'{A}{i}', (Ai, Xi)))
          Ai = f'{A}{i}'
      prods.add(Production(A, (Ai, α[-1])))
    else:
      prods.add(Production(A, α))
  return Grammar(G.N | {A for A, α in prods}, G.T, prods, G.S)

In [ ]:
# esempio d'uso

U = Grammar.from_string("""
S -> A B C D
""")

make_binary(U).P

#### Di nuovo, l'esempio più elaborato

In [ ]:
# fig. 4.15, pag. 123 (rinominando alcuni non terminali)

G_cnf = make_binary(transform_nonsolitary(Gp_clean))
G_cnf.P

## Parsing CYK con  G in forma CNF

In [ ]:
# fig. 4.16, pag. 123

INPUT = '32.5e+1'
R = cyk(G_cnf, INPUT)
R

## Una derivazione leftmost

Per ottenere una derivazione *leftmost* ragioniamo come per la funzione `fake_parse` della precedente sezione, ma invece di restituire un albero restituiamo l'indice della produzione in gioco (ottenuto invocando `G.P.index`)

In [ ]:
def get_leftmost_prods(G, R, INPUT):
  def prods(X, i, l):
    if l == 1:
      return [G.P.index(Production(X, (INPUT[i - 1],)))]
    for A, (B, C) in filter(Production.such_that(lhs = X, rhs_len = 2), G.P):
      for k in range(1, l):
        if B in R[i, k] and C in R[i + k, l - k]:
          return [G.P.index(Production(A, (B, C)))] + prods(B, i, k) + prods(C, i + k, l - k)
  return prods(G.S, 1, len(INPUT))

In [ ]:
leftmost_prods = get_leftmost_prods(G_cnf, R, INPUT)
leftmost_prods

In [ ]:
d = Derivation(G_cnf).leftmost(leftmost_prods)
d

In [ ]:
ProductionGraph(d)

Come si possono ottenere le produzioni di una derivazione *rightomst*?

## Il parsing in G (non in CNF)? (Sez. 4.2.6)

Un primo requisito, per poter effettuare il parsing secondo le produzioni della grammatica originale è non eliminare i simboli improduttivi e irraggiungibili.

In [ ]:
# Non effettuiamo la pulizia

# Gp = eliminate_unit_rules(eliminate_ε_rules(G))
# Gp_clean = remove_unproductive_unreachable(Gp)
# G_cnf = make_binary(transform_nonsolitary(Gp_clean))

Gp_cnf = make_binary(transform_nonsolitary(Gp))
Gp_cnf.P

Ora calcoliamo la tabella `Rp` e la completiamo con le ε-regole

In [ ]:
# Otteniamo Rp tramite il parsing rispetto alla grammatica non ripulita Gp_cnf

INPUT = '32.5e+1'
Rp = cyk(Gp_cnf, INPUT)
Rp

In [ ]:
# Calcoliamo l'insieme dei simboli A tali che A -> ε

Rε = {A for A in Gp_cnf.N if (ε, ) in Gp_cnf.alternatives(A)}

In [ ]:
# Li aggiungiamo in fondo alla tabella

for i in range(1, len(INPUT) + 2): Rp[i, 0] = Rε

In [ ]:
Rp

## Ricostruzione del (vero) albero di parsing

Ora costruiamo (per una assegnata tabella `R` e un assegnato `INPUT` la funzione `derives(ω, i, l)` che
(in base alle informaizoni nella tabella) se la forma sentenziale corispondete al primo argomento deriva la sottoparola $s_{i,l}$ dell'input che inizia dall'`i`-esimo simbolo ed è lunga `l` restituisce un elenco di lungezze, ciascuna delle quali corrisponde a quanti simboli della sottoparola derivano da ciascun non terminale (o corrispondono ai terminali) in `ω`, oppure `None` se `ω` non deriva $s_{i,l}$.

In [ ]:
def make_derives(R, INPUT):
  def derives(ω, i, l):
    if not ω or (ε, ) == ω: 
      return [] if l == 0 else None
    X, *χ = ω
    if X in G.T:
      if i <= len(INPUT) and X == INPUT[i - 1]:
        s = derives(χ, i + 1, l - 1)
        if s is not None: return [1] + s
    else:
      for k in range(0, l + 1):
        if X in R[i, k]:
          s = derives(χ, i + k, l - k)
          if s is not None: return [k] + s
    return None
  return derives

In [ ]:
# costruiamo derive sulla tabella ed input precedenti

derives = make_derives(Rp, INPUT)

In [ ]:
# una prova di esecuzione 

# INPUT = '32.5e+1'

derives(['Integer', 'Fraction', 'Scale'], 1, len(INPUT))

I tre non terminali (che sono il lato destro della produzione `Real -> Integer Fraction Scale`) producono l'input (ossia tutti e 7 i suoi simboli a partire dal primo) e più precisamente:

- `Integer` produrrà '35' (una sottoparola lunga 2),
- `Fraction` produrrà '.5' (una sottoparola lunga 2),
- `Scale` produrrà 'e+1', (una sottoparola lunga 3).

Una volta scritta la suddetta funzione basta seguire l'algoritmo descritto a partire da pag. 115.

In [ ]:
def get_original_leftmost_prods(G, derives, N):
  def prods(X, i, l):
    if X in G.T: return []
    for A, α in filter(Production.such_that(lhs = X), G.P):
      d = derives(α, i, l)
      if d is None: continue
      res = [G.P.index(Production(A, α))]
      for B, l in zip(α, d): 
        res.extend(prods(B, i, l))
        i += l
      return res
  return prods(G.S, 1, N)

Ora possiamo costruire (le produzioni, la derivazione e) l'albero nella grammatica originale!

In [ ]:
# le produzioni

leftmost_prods = get_original_leftmost_prods(G, derives, len(INPUT))
leftmost_prods

In [ ]:
# la derivazione

d = Derivation(G).leftmost(leftmost_prods)
d

In [ ]:
# l'albero

ProductionGraph(d)

### E le ε-produzioni?

In [ ]:
INPUT = tuple('32.5')

In [ ]:
Rp = cyk(Gp_cnf, INPUT)
Rε = {A for A in Gp_cnf.N if (ε, ) in Gp_cnf.alternatives(A)}
for i in range(1, len(INPUT) + 2): Rp[i, 0] = Rε

In [ ]:
leftmost_prods = get_original_leftmost_prods(G, make_derives(Rp, INPUT), len(INPUT))

In [ ]:
ProductionGraph(Derivation(G).leftmost(leftmost_prods))    